In [1]:
import os
import pandas as pd
import numpy as np

sales_train_validation = pd.read_csv('sales_train_validation.csv')
sales_train_validation = pd.DataFrame(sales_train_validation ,columns=['id','item_id','dept_id','cat_id','store_id','state_id'])

In [2]:
for f in os.listdir(r'prediction'): 
    df1 = pd.read_csv('prediction/'+f)
    df1 = pd.merge(df1, sales_train_validation, on='id', how='left')    
    df1 = pd.concat( [df1,pd.DataFrame(np.ones((df1.shape[0],1)))], axis=1)  
    
    # level1
    df2 = df1.groupby(0).sum()    
    # level2
    df2 = pd.concat( [df2, df1.groupby("state_id").sum().iloc[:,:-1]] , axis=0)    
    # level3
    df2 = pd.concat( [df2, df1.groupby("store_id").sum().iloc[:,:-1]] , axis=0)       
    # level4
    df2 = pd.concat( [df2, df1.groupby("cat_id").sum().iloc[:,:-1]] , axis=0)  
    # level5
    df2 = pd.concat( [df2, df1.groupby("dept_id").sum().iloc[:,:-1]] , axis=0)     
    # level6
    df2 = pd.concat( [df2, df1.groupby(['state_id',"cat_id"]).sum().iloc[:,:-1]] , axis=0)   
    # level7
    df2 = pd.concat( [df2, df1.groupby(['state_id',"dept_id"]).sum().iloc[:,:-1]] , axis=0)   
    # level8
    df2 = pd.concat( [df2, df1.groupby(['store_id',"cat_id"]).sum().iloc[:,:-1]] , axis=0)   
    # level9
    df2 = pd.concat( [df2, df1.groupby(['store_id',"dept_id"]).sum().iloc[:,:-1]] , axis=0)   
    # level10
    df2 = pd.concat( [df2, df1.groupby(['item_id']).sum().iloc[:,:-1]] , axis=0)     
    # level11
    df2 = pd.concat( [df2, df1.groupby(['state_id','item_id']).sum().reindex(['WI', 'CA', 'TX'], level='state_id').iloc[:,:-1]] , axis=0)   
    # level12
    df2 = pd.concat( [df2, df1.groupby(['store_id','item_id']).sum().iloc[:,:-1] ] , axis=0) 
    
    # print(df2.shape)
    df2.to_csv('prediction_groupby/{}.csv'.format(f), index=False)


In [3]:
lst = []

for f in os.listdir(r'prediction_groupby'): #path
    df3 = pd.read_csv('prediction_groupby/'+f)
    lst.append(np.array(df3))

q_005,q_025,q_165,q_25,q_5,q_75,q_835,q_975,q_995 = np.zeros(df3.shape),np.zeros(df3.shape),np.zeros(df3.shape),np.zeros(df3.shape),np.zeros(df3.shape),np.zeros(df3.shape),np.zeros(df3.shape),np.zeros(df3.shape),np.zeros(df3.shape)

for i in range(lst[0].shape[0]):
    for j in range(lst[0].shape[1]):
        qqu = []
        for f in lst:
            qqu.append(f[i][j])
        q_005[i][j] = pd.Series(qqu).quantile(0.005)
        q_025[i][j] = pd.Series(qqu).quantile(0.025)
        q_165[i][j] = pd.Series(qqu).quantile(.165)
        q_25[i][j] = pd.Series(qqu).quantile(.25)
        q_5[i][j] = pd.Series(qqu).quantile(.5)
        q_75[i][j] = pd.Series(qqu).quantile(.75)
        q_835[i][j] = pd.Series(qqu).quantile(.835)
        q_975[i][j] = pd.Series(qqu).quantile(.975)
        q_995[i][j] = pd.Series(qqu).quantile(.995) 

In [23]:
submit = np.concatenate((q_005,q_025,q_165,q_25,q_5,q_75, q_835,q_975,q_995),axis=0)
submission = pd.DataFrame(data = np.array(submit))  
submission = pd.concat((submission, submission), ignore_index=True)  
print(submission.shape)

sample_submission = pd.read_csv("sample_submission.csv")  
idColumn = sample_submission[["id"]]
submission[["id"]] = idColumn  
cols = list(submission.columns)
cols = cols[-1:] + cols[:-1] 
submission = submission[cols]
colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]
submission.columns = colsdeneme
submission.to_csv("test_submission.csv", index=False)

(771120, 28)
